In [ ]:
import os
import requests
import streamlit as st
from time import sleep
from langchain_openai import OpenAI, ChatOpenAI
from langchain_chroma import Chroma
from langchain.chains import ConversationalRetrievalChain, LLMChain, RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools import tool
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, AgentType
from langchain_community.utilities.openweathermap import OpenWeatherMapAPIWrapper
from dotenv import load_dotenv, find_dotenv
from langchain.tools import Tool
from io import BytesIO
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.utils import simpleSplit

# 📌 Load API keys from environment variables
_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 📌 Initialize embedding model and LLM
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=OPENAI_API_KEY)
llm = ChatOpenAI(api_key=OPENAI_API_KEY, temperature=0, model="gpt-4-turbo")

# 📌 Load the vector database
vectorstore = Chroma(
    collection_name="landmarks_rag",
    embedding_function=embedding_model
)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# 📌 Define a custom prompt for the chatbot
rag_prompt = PromptTemplate(
    input_variables=["context", "question"],  
    template="""
You are a Puerto Rico travel assistant. 
Use the retrieved information to provide the best travel recommendations in a natural and engaging way.
First Welcome the user and ask for their preferences and details that will help you to provide the best recommendations. \
ENSURE to ask as much information as possible to provide the best recommendations. ALWAYS ask follow-up questions to lock user's selections. \
GUIDE the user through the process of selecting the best landmarks based on their preferences. \

Questions you can ask:
- How many days do you plan to stay?
- What type of activities do you enjoy?
- Are you traveling with family or alone?
- Do you have any specific dietary restrictions?
- Do you prefer outdoor or indoor activities?

Follow-up questions:
- Which of these landmarks are you most interested in?
- Are you interested in historical sites or natural landmarks?
- Do you prefer adventurous activities or relaxing experiences?
- Do you want to include any of these landmarks in your itinerary?
- Would you like to visit any of these landmarks?

### Context (Relevant Information from RAG):
{context}

### Instructions:
- Provide a well-structured travel recommendation based on the retrieved landmarks.
- Ensure continuity with previous discussions.
- Prioritize landmarks that match the user’s preferences.
- If multiple options exist, suggest the BEST ones with reasoning.
- Avoid repeating information already given in the conversation.
- In the end ask the user which of these locations you will like to visit.

### User Question:
{question}

### AI Response:
Based on the information available, here’s what I recommend:

"""
)

# 📌 Configure the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": rag_prompt}
)

# 📌 Apply UI settings
st.set_page_config(page_title="AI Travel2PR Planner", page_icon="🌍", layout="centered")

# 📌 Display a header
st.markdown("<h1>🗺️ AI Travel to Puerto Rico Planner 🏝️</h1>", unsafe_allow_html=True)
st.markdown("<p>Ask for recommendations about places to visit in Puerto Rico!</p>", unsafe_allow_html=True)

# 📌 Initialize session state
if "messages" not in st.session_state:
    st.session_state["messages"] = []
if "itinerary_data" not in st.session_state:
    st.session_state["itinerary_data"] = []

# 📌 Function to reset conversation
def reset_conversation():
    st.session_state["messages"] = []
    st.session_state["user_input"] = ""
    st.session_state.pop("final_itinerary", None)
    st.session_state["itinerary_data"] = []  # Clear itinerary data
    st.rerun()

st.button("🔄 Start a New Consultation", on_click=reset_conversation)

# 📌 Function to process user input
def process_input():
    user_query = st.session_state["user_input"].strip()
    
    if user_query:
        with st.spinner("🔎 Searching for travel recommendations..."):
            response = qa_chain.run(user_query)
        
        # Append user message
        st.session_state["messages"].append({"role": "user", "content": user_query})
        st.session_state["messages"].append({"role": "assistant", "content": response})
        
        # Store relevant places
        st.session_state["itinerary_data"].append(response) 
        
        # Clear input field
        st.session_state.update({"user_input": ""})
        st.rerun()

# 📌 Display conversation history
for message in st.session_state["messages"]:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# 📌 User input field below chat messages
st.text_input("✍️ Type your question and press Enter:", "", key="user_input", on_change=process_input)

# 📌 Generate formatted PDF itinerary
def generate_pdf(itinerary_text):
    if not itinerary_text.strip():
        return None  
    
    buffer = BytesIO()
    c = canvas.Canvas(buffer, pagesize=letter)
    c.setFont("Helvetica-Bold", 14)
    c.drawString(200, 750, "Puerto Rico Travel Itinerary")
    c.setFont("Helvetica", 12)
    
    y_position = 700
    for line in itinerary_text.split("\n"):
        wrapped_lines = simpleSplit(line, "Helvetica", 12, 500)
        for wrapped_line in wrapped_lines:
            c.drawString(50, y_position, wrapped_line)
            y_position -= 20  
            if y_position < 50:  
                c.showPage()
                c.setFont("Helvetica", 12)
                y_position = 750
    
    c.save()
    buffer.seek(0)
    return buffer

# 📌 Button to download the itinerary as a PDF
if st.button("📄 Download Itinerary as PDF"):
    if "itinerary_data" in st.session_state and st.session_state["itinerary_data"]:
        pdf_text = "\n".join(st.session_state["itinerary_data"])
        pdf_buffer = generate_pdf(pdf_text)
        st.download_button(
            label="📥 Click to Download",
            data=pdf_buffer,
            file_name="Puerto_Rico_Itinerary.pdf",
            mime="application/pdf"
        )
    else:
        st.warning("⚠️ No itinerary available to download. Ask the AI for recommendations first.")


In [ ]:
import os
import requests
import streamlit as st
from time import sleep
from langchain_openai import OpenAI, ChatOpenAI
from langchain_chroma import Chroma
from langchain.chains import ConversationalRetrievalChain, LLMChain, RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools import tool
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, AgentType
from langchain_community.utilities.openweathermap import OpenWeatherMapAPIWrapper
from dotenv import load_dotenv, find_dotenv
from langchain.tools import Tool
from io import BytesIO
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.utils import simpleSplit

# 📌 Load API keys from environment variables
_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")

# 📌 Initialize embedding model and LLM
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=OPENAI_API_KEY)
llm = ChatOpenAI(api_key=OPENAI_API_KEY, temperature=0, model="gpt-4-turbo")

# 📌 Load the vector database
vectorstore = Chroma(
    collection_name="landmarks_rag",
    embedding_function=embedding_model
)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# 📌 Memory Tool for Tracking User Preferences
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 📌 Define a custom prompt for the chatbot
rag_prompt = PromptTemplate(
    input_variables=["context", "question", "chat_history"],  
    template="""
You are a Puerto Rico travel assistant. 
Use the retrieved information to provide the best travel recommendations in a natural and engaging way.
First Welcome the user and ask for their preferences and details that will help you to provide the best recommendations. \
ENSURE to ask as much information as possible to provide the best recommendations. ALWAYS ask follow-up questions to lock user's selections. \
GUIDE the user through the process of selecting the best landmarks based on their preferences. \

### User Preferences Stored:
{chat_history}

### Context (Relevant Information from RAG):
{context}

### Instructions:
- Provide a well-structured travel recommendation based on the retrieved landmarks.
- Ensure continuity with previous discussions.
- Prioritize landmarks that match the user’s preferences.
- If multiple options exist, suggest the BEST ones with reasoning.
- Avoid repeating information already given in the conversation.
- In the end ask the user which of these locations you will like to visit.

### User Question:
{question}

### AI Response:
Based on the information available, here’s what I recommend:

"""
)

# 📌 Configure the RetrievalQA chain with Memory
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": rag_prompt, "memory": memory}
)

# 📌 Apply UI settings
st.set_page_config(page_title="AI Travel2PR Planner", page_icon="🌍", layout="centered")

# 📌 Display a header
st.markdown("<h1>🗺️ AI Travel to Puerto Rico Planner 🏝️</h1>", unsafe_allow_html=True)
st.markdown("<p>Ask for recommendations about places to visit in Puerto Rico!</p>", unsafe_allow_html=True)

# 📌 Initialize session state
if "messages" not in st.session_state:
    st.session_state["messages"] = []
if "itinerary_data" not in st.session_state:
    st.session_state["itinerary_data"] = []
if "user_preferences" not in st.session_state:
    st.session_state["user_preferences"] = []

# 📌 Function to reset conversation
def reset_conversation():
    st.session_state["messages"] = []
    st.session_state["user_input"] = ""
    st.session_state.pop("final_itinerary", None)
    st.session_state["itinerary_data"] = []  # Clear itinerary data
    st.session_state["user_preferences"] = []  # Clear stored user preferences
    memory.clear()  # Clear memory tool
    st.rerun()

st.button("🔄 Start a New Consultation", on_click=reset_conversation)

# 📌 Function to fetch weather for selected locations
def get_weather(location):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={WEATHER_API_KEY}&units=metric"
    response = requests.get(url).json()
    if response.get("weather"):
        return f"{response['weather'][0]['description']}, Temp: {response['main']['temp']}°C"
    return "Weather data unavailable"

# 📌 Function to process user input
def process_input():
    user_query = st.session_state["user_input"].strip()
    
    if user_query:
        with st.spinner("🔎 Searching for travel recommendations..."):
            response = qa_chain.invoke({"question": user_query})
        
        # Append user message
        st.session_state["messages"].append({"role": "user", "content": user_query})
        st.session_state["messages"].append({"role": "assistant", "content": response["result"]})  # Extract only the text response
        
        # Store relevant places
        st.session_state["itinerary_data"].append(response["result"]) 
        
        # Capture user preferences for future queries
        st.session_state["user_preferences"].append(user_query)
        memory.save_context({"input": user_query}, {"output": response["result"]})
        
        # Clear input field
        st.session_state.update({"user_input": ""})
        st.rerun()

# 📌 Display conversation history
for message in st.session_state["messages"]:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# 📌 User input field below chat messages
st.text_input("✍️ Type your question and press Enter:", "", key="user_input", on_change=process_input)

# 📌 Generate formatted PDF itinerary
def generate_pdf(itinerary_text):
    if not itinerary_text.strip():
        return None  
    
    buffer = BytesIO()
    c = canvas.Canvas(buffer, pagesize=letter)
    c.setFont("Helvetica-Bold", 14)
    c.drawString(200, 750, "Puerto Rico Travel Itinerary")
    c.setFont("Helvetica", 12)
    
    y_position = 700
    for line in itinerary_text.split("\n"):
        wrapped_lines = simpleSplit(line, "Helvetica", 12, 500)
        for wrapped_line in wrapped_lines:
            c.drawString(50, y_position, wrapped_line)
            y_position -= 20  
            if y_position < 50:  
                c.showPage()
                c.setFont("Helvetica", 12)
                y_position = 750
    
    c.save()
    buffer.seek(0)
    return buffer

# 📌 Button to download the itinerary as a PDF
if st.button("📄 Download Itinerary as PDF"):
    if "itinerary_data" in st.session_state and st.session_state["itinerary_data"]:
        pdf_text = "\n".join(st.session_state["itinerary_data"])
        pdf_buffer = generate_pdf(pdf_text)
        st.download_button(
            label="📥 Click to Download",
            data=pdf_buffer,
            file_name="Puerto_Rico_Itinerary.pdf",
            mime="application/pdf"
        )
    else:
        st.warning("⚠️ No itinerary available to download. Ask the AI for recommendations first.")

2025-02-13 20:13:03.134 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 20:13:03.139 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 20:13:03.346 
  command:

    streamlit run c:\Users\larry\anaconda3\envs\travel2pr_ai\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-02-13 20:13:03.347 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 20:13:03.348 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 20:13:03.349 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 20:13:03.350 Thread 'MainThread': missing ScriptRunContext! This warning can be

# **Standalone Testing Script**

In [ ]:
import os
import requests
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv, find_dotenv

# 📌 Load API keys
_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")

# 📌 Initialize models and vectorstore
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=OPENAI_API_KEY)
llm = ChatOpenAI(api_key=OPENAI_API_KEY, temperature=0, model="gpt-4-turbo")
vectorstore = Chroma(collection_name="landmarks_rag", embedding_function=embedding_model)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# 📌 Memory tool to retain user preferences
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 📌 Define a custom prompt template
rag_prompt = PromptTemplate(
    input_variables=["context", "question", "chat_history"],  
    template="""
You are a Puerto Rico travel assistant. 
Use the retrieved information to provide the best travel recommendations in a natural and engaging way.
Ensure you ask follow-up questions to refine the user's itinerary.

### User Preferences Stored:
{chat_history}

### Context (Relevant Information from RAG):
{context}

### Instructions:
- Provide structured travel recommendations.
- Retain user preferences and integrate them into responses.
- Prioritize highly relevant landmarks.
- Suggest only the best options.

### User Question:
{question}

### AI Response:
Based on the information available, here’s what I recommend:
"""
)

# 📌 Configure the RetrievalQA chain with memory
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": rag_prompt, "memory": memory}
)

# 📌 Function to fetch weather data
def get_weather(location):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={WEATHER_API_KEY}&units=metric"
    response = requests.get(url).json()
    if response.get("weather"):
        return f"{response['weather'][0]['description']}, Temp: {response['main']['temp']}°C"
    return "Weather data unavailable"

# 📌 Simulated User Conversation (Test Cases)
def test_chatbot():
    print("\n🛠️ Running Chatbot V2.2 Test...\n")

    # Step 1: User starts the conversation
    user_input = "Hello"
    print(f"\nUser: {user_input}")
    response = qa_chain.run(user_input)
    print(f"\nAI: {response}\n")

    # Step 2: User provides trip details
    user_input = "I will be staying in Puerto Rico for 5 days."
    print(f"\nUser: {user_input}")
    response = qa_chain.run(user_input)
    print(f"\nAI: {response}\n")

    # Step 3: User selects activities
    user_input = "I like hiking and cultural activities. Traveling with my family."
    print(f"\nUser: {user_input}")
    response = qa_chain.run(user_input)
    print(f"\nAI: {response}\n")

    # Step 4: AI recommends landmarks
    user_input = "That sounds great. Show me more places to visit."
    print(f"\nUser: {user_input}")
    response = qa_chain.run(user_input)
    print(f"\nAI: {response}\n")

    # Step 5: User confirms landmark selection
    user_input = "I would like to visit El Yunque and Old San Juan."
    print(f"\nUser: {user_input}")
    response = qa_chain.run(user_input)
    print(f"\nAI: {response}\n")

    # Step 6: Fetch weather for selected landmarks
    print("\n📍 Fetching weather data for El Yunque and Old San Juan...")
    weather_yunque = get_weather("El Yunque, PR")
    weather_sanjuan = get_weather("San Juan, PR")
    print(f"\n🌦️ El Yunque: {weather_yunque}")
    print(f"🌦️ Old San Juan: {weather_sanjuan}\n")

    # Step 7: User requests final itinerary
    user_input = "Please generate my final itinerary."
    print(f"\nUser: {user_input}")
    response = qa_chain.run(user_input)
    print(f"\nAI: {response}\n")

    print("\n✅ Chatbot V2.2 Test Completed Successfully!\n")

# 📌 Run the test
if __name__ == "__main__":
    test_chatbot()
    

🗺️ AI Travel Planner - Test Mode 🏝️
Type 'exit' to stop.



ValueError: One input key expected got ['question', 'input_documents']